In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.formula.api as smf
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import ElasticNet, Ridge,LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [11]:
players_df = pd.read_csv('data/players/players_summary.csv')
fixtures_df = pd.read_csv('data/fixture_data.csv')
fdr_df = pd.read_csv('data/fdr.csv')

In [64]:
def combine_data(players, fixtures, fdr):
    fixtures = fixtures.loc[fixtures['game_played'], ['Wk', 'game_id','Home', 'Away']]
    fixtures_home = fixtures.copy().rename(columns={'Home': 'Team'}).merge(
        fdr, how='left', on='Team').drop(columns=['Away'])
    fixtures_home['home'] = True
    fixtures_away = fixtures.copy().rename(columns={'Away': 'Team'}).merge(
        fdr, how='left', on='Team').drop(columns=['Home'])
    fixtures_away['home'] = False
    fixtures_fdr = pd.concat([fixtures_home, fixtures_away], ignore_index=True)
    players_fdr = players.merge(fixtures_fdr, how='left', on=['game_id', 'home'])
    return players_fdr

def generate_ts_data(df, N):
    top_players = df.groupby('Player', as_index=False).sum()
    top_players = top_players[['Player', 'xG']]
    top_players = top_players.sort_values('xG', ascending=False)
    top_players = top_players.head(N)['Player'].to_list()
    ts_data = df.loc[df['Player'].isin(top_players), ['Player', 'Wk', 'xG']]
    
    all_weeks = range(df['Wk'].min(), df['Wk'].max() + 1)
    all_players = ts_data['Player'].unique()
    full_index = pd.MultiIndex.from_product([all_players, all_weeks], names=['Player', 'Wk'])
    ts_data = ts_data.set_index(['Player', 'Wk']).reindex(full_index, fill_value=0).reset_index()
    return ts_data

In [88]:
N=10
players_fdr = combine_data(players_df, fixtures_df, fdr_df)
ts_data = generate_ts_data(players_fdr, N)

ts_data.sample(5)

,Player,Wk,xG
30,Thiago,7,0.5
0,Antoine Semenyo,1,0.9
22,Erling Haaland,7,0.2
31,Thiago,8,0.8
18,Erling Haaland,3,1.4


In [89]:
np.random.seed(42)
groups = ts_data['Player'].nunique()
periods = ts_data['Wk'].nunique()
df = ts_data.rename(columns={'Player': 'group', 'Wk': 'time', 'xG': 'variable'})

In [92]:
import pandas as pd
import altair as alt

def create_rolling_predictions(df, window=3):
    """Create rolling average predictions for all players"""
    model_df = df.copy()
    
    # Calculate rolling average for each group
    ts_dict = {}
    for group in model_df['group'].unique():
        group_data = model_df[model_df['group'] == group].copy()
        ts = group_data['variable']
        
        # Calculate rolling average
        rolling_avg = ts.rolling(window=window).mean()
        
        # Get last rolling average as first prediction
        predictions = []
        pred = rolling_avg.iloc[-1]
        predictions.append(pred)
        
        # Predict next 3 periods
        values = ts.tolist()
        for _ in range(2, 4):
            last_two = values[-2:]
            new_pred = (last_two[0] + last_two[1] + predictions[-1]) / 3
            predictions.append(new_pred)
            values.append(new_pred)
            
        ts_dict[group] = {
            'history': ts,
            'predictions': predictions
        }
    
    # Create plotting dataframe
    plot_data = []
    
    for group, data in ts_dict.items():
        # Historical data
        history_df = pd.DataFrame({
            'time': range(1, len(data['history']) + 1),
            'value': data['history'].values,
            'type': 'Actual',
            'group': group
        })
        
        # Future predictions
        future_times = range(len(data['history']) + 1, len(data['history']) + 4)
        future_df = pd.DataFrame({
            'time': future_times,
            'value': data['predictions'],
            'type': 'Prediction',
            'group': group
        })
        
        plot_data.append(history_df)
        plot_data.append(future_df)
    
    return pd.concat(plot_data, ignore_index=True)

# Create predictions for all players
plot_df = create_rolling_predictions(df)

# Create interactive chart
base = alt.Chart(plot_df).encode(
    x=alt.X('time:O', title='Time Period'),
    y=alt.Y('value:Q', title='Value', scale=alt.Scale(domain=[0, 2])),
    color=alt.Color('type:N', title='Type'),
    tooltip=['group', 'time', 'value', 'type']
)

# Lines
lines = base.mark_line()

# Points
points = base.mark_circle(size=60)

# Combine
chart = (lines + points).facet(
    facet=alt.Facet('group:N', title='Player'),
    columns=2
).properties(
    title='Rolling Average Forecasts by Player'
).configure_title(
    fontSize=20
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
)

# Display chart
chart

alt.FacetChart(...)

In [ ]:
N=3
df['rolling_mean'] = df.groupby('group')['variable'].transform(lambda x: x.rolling(window=N, min_periods=1).mean())
# Show rolling mean for all periods
print(df)

# Predict for future periods 9, 10, 11 using last available rolling mean per group
future_periods = [9, 10, 11]
future_preds = []

for group, group_df in df.groupby('group'):
    # Recalculate rolling mean for the next period using last N values
    for t in future_periods:
        last_rolling = group_df['rolling_mean'].iloc[-1]
        last_N_values = group_df['variable'].iloc[-N:]
        next_rolling = last_N_values.mean()
        last_rolling = next_rolling
        future_preds.append({'group': group, 'time': t, 'predicted_rolling_mean': last_rolling})

future_preds_df = pd.DataFrame(future_preds)
future_preds_df

              group  time  variable  rolling_mean
0      Hugo Ekitike     1       1.1      1.100000
1      Hugo Ekitike     2       0.2      0.650000
2      Hugo Ekitike     3       0.0      0.433333
3      Hugo Ekitike     4       0.1      0.100000
4      Hugo Ekitike     5       0.3      0.133333
..              ...   ...       ...           ...
235  Nick Woltemade     4       0.3      0.100000
236  Nick Woltemade     5       0.0      0.100000
237  Nick Woltemade     6       0.3      0.200000
238  Nick Woltemade     7       1.4      0.566667
239  Nick Woltemade     8       0.7      0.800000

[240 rows x 4 columns]


,group,time,predicted_rolling_mean
0,Amad Diallo,9,0.200000
1,Amad Diallo,10,0.200000
2,Amad Diallo,11,0.200000
3,Antoine Semenyo,9,0.366667
4,Antoine Semenyo,10,0.366667
...,...,...,...
85,Viktor Gyökeres,10,0.433333
86,Viktor Gyökeres,11,0.433333
87,Yankuba Minteh,9,0.166667
88,Yankuba Minteh,10,0.166667


## Statistical Methods

In [ ]:
import pandas as pd
import altair as alt
import numpy as np

# Create rolling average predictions for different window sizes
def create_rolling_predictions(df, group_col='group', time_col='time', target_col='variable', max_window=5):
    predictions_df = []
    
    for group in df[group_col].unique():
        group_data = df[df[group_col] == group].copy().sort_values(time_col)
        
        # Create predictions for each window size
        for window in range(1, max_window + 1):
            temp_df = group_data.copy()
            temp_df['prediction'] = temp_df[target_col].rolling(window=window).mean()
            temp_df['window_size'] = window
            predictions_df.append(temp_df)
    
    return pd.concat(predictions_df, ignore_index=True)

# Create visualization
def plot_rolling_predictions(predictions_df):
    base = alt.Chart(predictions_df).encode(
        x=alt.X('time:Q', title='Time Period'),
        color=alt.Color('group:N', title='Player')
    )
    
    # Actual values line
    actual = base.mark_line().encode(
        y=alt.Y('variable:Q', title='Value'),
    ).properties(
        title='Actual vs Rolling Average Predictions'
    )
    
    # Predictions line
    predicted = base.mark_line(strokeDash=[5,5]).encode(
        y=alt.Y('prediction:Q'),
    )
    
    # Create sliders
    window_slider = alt.binding_range(min=1, max=5, step=1, name='Window Size: ')
    player_slider = alt.binding_range(min=1, max=len(predictions_df['group'].unique()), step=1, name='Number of Players: ')
    
    # Create selections
    select_window = alt.selection_point(name="window_select", 
                                       fields=['window_size'],
                                       bind=window_slider,
                                       value=1)
    
    select_players = alt.selection_point(name="player_select",
                                        fields=['player_rank'],
                                        bind=player_slider,
                                        value=5)
    
    # Add player ranking for filtering
    player_totals = predictions_df.groupby('group')['variable'].sum().reset_index()
    player_totals['player_rank'] = player_totals['variable'].rank(ascending=False)
    predictions_df = predictions_df.merge(player_totals[['group', 'player_rank']], 
                                        on='group')
    
    # Combine charts with both filters
    chart = (actual + predicted).add_params(
        select_window,
        select_players
    ).transform_filter(
        select_window
    ).transform_filter(
        f"datum.player_rank <= {select_players.name}.player_rank"
    ).properties(
        width=600,
        height=400
    )
    
    return chart

# Generate predictions
rolling_predictions = create_rolling_predictions(df)

# Create and display chart
chart = plot_rolling_predictions(rolling_predictions)
chart

alt.LayerChart(...)

In [73]:
# ============================================================================
# APPROACH 1: STACKING - Use all data points
# ============================================================================

# Create lagged features for each group separately
df_features = []
for g in df['group'].unique():
    group_df = df[df['group'] == g].copy().sort_values('time')
    group_df['lag1'] = group_df['variable'].shift(1)
    group_df['lag2'] = group_df['variable'].shift(2)
    group_df['time_linear'] = group_df['time']
    df_features.append(group_df)

df_stacked = pd.concat(df_features, ignore_index=True)
df_stacked = df_stacked.dropna()  # Remove rows with NaN from lagging

# Simple AR model using all data
X = df_stacked[['lag1', 'time_linear']]
y = df_stacked['variable']

# Fit AR model
ar_model = LinearRegression()
ar_model.fit(X, y)

predictions = ar_model.predict(X)
stacked_mse = mean_squared_error(y, predictions)
stacked_rmse = np.sqrt(stacked_mse)

# ============================================================================
# APPROACH 2: STACKED ARIMA
# ============================================================================
"""
Stacked ARIMA model: Fit ARIMA on each group and use average parameters for the model predictor.
"""

# Fit ARIMA on each group and collect parameters
arima_params = []
group_mse = []

for g in df['group'].unique():
    group_data = df[df['group'] == g].sort_values('time')['variable'].values
    try:
        model = ARIMA(group_data, order=(1, 0, 0))
        fit = model.fit()
        arima_params.append({
            'group': g,
            'ar_coef': fit.params[1],
            'intercept': fit.params[0],
            'sigma': np.sqrt(fit.params[2])
        })
        fitted = fit.fittedvalues
        mse = mean_squared_error(group_data[1:], fitted[1:])
        group_mse.append(mse)
    except:
        pass

params_df = pd.DataFrame(arima_params)

# Use average parameters for new groups
avg_ar = params_df['ar_coef'].mean()
avg_intercept = params_df['intercept'].mean()

print(f"Average AR Coefficient: {avg_ar}")
print(f"Average Intercept: {avg_intercept}")

print(params_df)

# ============================================================================
# APPROACH 3: MIXED EFFECTS MODEL (Group Random Effects)
# ============================================================================

# ============================================================================
# APPROACH 4: TIME SERIES CROSS-VALIDATION
# ============================================================================

# ============================================================================
# SUMMARY AND RECOMMENDATIONS
# ============================================================================


Average AR Coefficient: -0.10099312358157272
Average Intercept: 0.3639908350879055
                    group       ar_coef  intercept     sigma
0            Hugo Ekitike  1.756104e-01   0.255825  0.336989
1              Cody Gakpo  6.978591e-01   0.540943  0.383441
2           Mohamed Salah -3.484976e-01   0.309906  0.223342
3         Antoine Semenyo -3.871776e-01   0.482807  0.298194
4            Jacob Murphy  3.373516e-07   0.199996  0.239789
5             Richarlison -2.414776e-01   0.192328  0.311799
6          Jaidon Anthony  4.559239e-01   0.251669  0.224430
7           Danny Welbeck -4.561135e-01   0.394930  0.474255
8          Yankuba Minteh  9.414468e-02   0.209645  0.213579
9          Erling Haaland -3.799658e-01   1.070424  0.567087
10             Chris Wood -2.491116e-01   0.318436  0.312073
11                 Thiago  3.518650e-01   0.464778  0.317552
12           Kevin Schade  2.162983e-01   0.258461  0.307462
13        Estêvão Willian  4.477840e-02   0.212350  0.153422
14

In [ ]:
import altair as alt
import pandas as pd
from typing import Dict, List

class ModelVisualizer:
    """Simple model comparison visualization using Altair"""
    
    def plot_model_comparison(self, metrics: Dict[str, Dict[str, float]], 
                            save_path: str = 'model_comparison.html'):
        """Create interactive comparison chart of model metrics"""
        
        # Prepare data
        df = pd.DataFrame([
            {
                'Model': model_name,
                'Metric': metric_name,
                'Value': value
            }
            for model_name, model_metrics in metrics.items()
            for metric_name, value in model_metrics.items()
        ])
        
        # Create chart
        chart = alt.Chart(df).mark_bar().encode(
            x='Model:N',
            y='Value:Q',
            color='Model:N',
            column='Metric:N',
            tooltip=['Model', 'Metric', 'Value']
        ).properties(
            width=120,
            title='Model Performance Comparison'
        ).interactive()
        
        # Save chart
        chart.save(save_path)
        return chart

    def plot_predictions(self, y_true: pd.Series, 
                        predictions: Dict[str, pd.Series],
                        save_path: str = 'predictions.html'):
        """Create scatter plot of actual vs predicted values"""
        
        # Prepare data
        df = pd.concat([
            pd.DataFrame({
                'Actual': y_true,
                'Predicted': pred,
                'Model': name
            })
            for name, pred in predictions.items()
        ])
        
        # Create chart
        chart = alt.Chart(df).mark_circle(size=60).encode(
            x='Actual:Q',
            y='Predicted:Q',
            color='Model:N',
            tooltip=['Model', 'Actual', 'Predicted']
        ).properties(
            width=500,
            height=400,
            title='Actual vs Predicted Values'
        ).interactive()
        
        # Add diagonal reference line
        diagonal = alt.Chart(pd.DataFrame({
            'x': [y_true.min(), y_true.max()],
            'y': [y_true.min(), y_true.max()]
        })).mark_line(color='red', strokeDash=[3,3]).encode(
            x='x',
            y='y'
        )
        
        final_chart = chart + diagonal
        final_chart.save(save_path)
        return final_chart
    
    
# After training models and getting predictions
visualizer = ModelVisualizer()

# Plot metric comparisons
metrics = {
    'Random Forest': {'RMSE': rf_test_rmse, 'R2': rf_test_r2},
    'Gradient Boosting': {'RMSE': gb_test_rmse, 'R2': gb_test_r2},
    'Ridge': {'RMSE': ridge_test_rmse, 'R2': ridge_test_r2},
    'Ensemble': {'RMSE': ensemble_rmse, 'R2': ensemble_r2}
}
visualizer.plot_model_comparison(metrics)

# Plot predictions
predictions = {
    'Random Forest': rf_pred_test,
    'Gradient Boosting': gb_pred_test,
    'Ridge': ridge_pred_test,
    'Ensemble': ensemble_pred_test
}
visualizer.plot_predictions(y_test, predictions)

## ML Approaches

In [ ]:
# ============================================================================
# FEATURE ENGINEERING
# ============================================================================

def create_features(df, max_lag=3):
    """Create lagged features and other time series features"""
    df_features = []
    
    for g in df['group'].unique():
        group_df = df[df['group'] == g].copy().sort_values('time')
        
        # Lagged values
        for i in range(1, max_lag + 1):
            group_df[f'lag{i}'] = group_df['variable'].shift(i)
        
        # Rolling statistics
        group_df['roll_mean_3'] = group_df['variable'].rolling(window=3).mean()
        group_df['roll_std_3'] = group_df['variable'].rolling(window=3).std()
        
        # Time features
        group_df['time'] = group_df['time']
        group_df['time_squared'] = group_df['time'] ** 2
        
        # Differences
        group_df['diff1'] = group_df['variable'].diff(1)
        group_df['diff2'] = group_df['variable'].diff(2)
        
        df_features.append(group_df)
    
    result = pd.concat(df_features, ignore_index=True)
    return result.dropna()

df_ml = create_features(df, max_lag=2)

# Prepare data
feature_cols = [col for col in df_ml.columns if col not in ['group', 'variable']]
X = df_ml[feature_cols]
y = df_ml['variable']

# Split train/test by time
train_mask = df_ml['time'] <= 6
X_train, X_test = X[train_mask], X[~train_mask]
y_train, y_test = y[train_mask], y[~train_mask]

print(f"\nTrain size: {len(X_train)}, Test size: {len(X_test)}")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ============================================================================
# MODEL 1: RANDOM FOREST
# ============================================================================

print("\n" + "="*70)
print("MODEL 1: RANDOM FOREST")
print("="*70)

rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=5,
    min_samples_split=5,
    random_state=42
)

rf_model.fit(X_train, y_train)
rf_pred_train = rf_model.predict(X_train)
rf_pred_test = rf_model.predict(X_test)

rf_train_rmse = np.sqrt(mean_squared_error(y_train, rf_pred_train))
rf_test_rmse = np.sqrt(mean_squared_error(y_test, rf_pred_test))
rf_test_r2 = r2_score(y_test, rf_pred_test)

print(f"Train RMSE: {rf_train_rmse:.4f}")
print(f"Test RMSE: {rf_test_rmse:.4f}")
print(f"Test R²: {rf_test_r2:.4f}")

# Feature importance
feature_imp = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 5 features:")
print(feature_imp.head().to_string(index=False))

# ============================================================================
# MODEL 2: GRADIENT BOOSTING
# ============================================================================

print("\n" + "="*70)
print("MODEL 2: GRADIENT BOOSTING")
print("="*70)

gb_model = GradientBoostingRegressor(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=42
)

gb_model.fit(X_train, y_train)
gb_pred_train = gb_model.predict(X_train)
gb_pred_test = gb_model.predict(X_test)

gb_train_rmse = np.sqrt(mean_squared_error(y_train, gb_pred_train))
gb_test_rmse = np.sqrt(mean_squared_error(y_test, gb_pred_test))
gb_test_r2 = r2_score(y_test, gb_pred_test)

print(f"Train RMSE: {gb_train_rmse:.4f}")
print(f"Test RMSE: {gb_test_rmse:.4f}")
print(f"Test R²: {gb_test_r2:.4f}")

# ============================================================================
# MODEL 3: RIDGE REGRESSION (Regularized Linear)
# ============================================================================

print("\n" + "="*70)
print("MODEL 3: RIDGE REGRESSION")
print("="*70)

ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train_scaled, y_train)
ridge_pred_train = ridge_model.predict(X_train_scaled)
ridge_pred_test = ridge_model.predict(X_test_scaled)

ridge_train_rmse = np.sqrt(mean_squared_error(y_train, ridge_pred_train))
ridge_test_rmse = np.sqrt(mean_squared_error(y_test, ridge_pred_test))
ridge_test_r2 = r2_score(y_test, ridge_pred_test)

print(f"Train RMSE: {ridge_train_rmse:.4f}")
print(f"Test RMSE: {ridge_test_rmse:.4f}")
print(f"Test R²: {ridge_test_r2:.4f}")

# Top coefficients
coef_df = pd.DataFrame({
    'feature': feature_cols,
    'coefficient': ridge_model.coef_
}).sort_values('coefficient', key=abs, ascending=False)

print("\nTop 5 coefficients:")
print(coef_df.head().to_string(index=False))

# ============================================================================
# MODEL 4: ELASTIC NET (L1 + L2 Regularization)
# ============================================================================

print("\n" + "="*70)
print("MODEL 4: ELASTIC NET")
print("="*70)

elastic_model = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
elastic_model.fit(X_train_scaled, y_train)
elastic_pred_train = elastic_model.predict(X_train_scaled)
elastic_pred_test = elastic_model.predict(X_test_scaled)

elastic_train_rmse = np.sqrt(mean_squared_error(y_train, elastic_pred_train))
elastic_test_rmse = np.sqrt(mean_squared_error(y_test, elastic_pred_test))
elastic_test_r2 = r2_score(y_test, elastic_pred_test)

print(f"Train RMSE: {elastic_train_rmse:.4f}")
print(f"Test RMSE: {elastic_test_rmse:.4f}")
print(f"Test R²: {elastic_test_r2:.4f}")

# Check sparsity
non_zero_coefs = np.sum(elastic_model.coef_ != 0)
print(f"Non-zero coefficients: {non_zero_coefs}/{len(feature_cols)}")

# ============================================================================
# MODEL 5: NEURAL NETWORK (MLP)
# ============================================================================

print("\n" + "="*70)
print("MODEL 5: NEURAL NETWORK (MLP)")
print("="*70)

mlp_model = MLPRegressor(
    hidden_layer_sizes=(32, 16),
    activation='relu',
    max_iter=1000,
    random_state=42,
    early_stopping=True
)

mlp_model.fit(X_train_scaled, y_train)
mlp_pred_train = mlp_model.predict(X_train_scaled)
mlp_pred_test = mlp_model.predict(X_test_scaled)

mlp_train_rmse = np.sqrt(mean_squared_error(y_train, mlp_pred_train))
mlp_test_rmse = np.sqrt(mean_squared_error(y_test, mlp_pred_test))
mlp_test_r2 = r2_score(y_test, mlp_pred_test)

print(f"Train RMSE: {mlp_train_rmse:.4f}")
print(f"Test RMSE: {mlp_test_rmse:.4f}")
print(f"Test R²: {mlp_test_r2:.4f}")

# ============================================================================
# MODEL COMPARISON
# ============================================================================

print("\n" + "="*70)
print("MODEL COMPARISON")
print("="*70)

comparison = pd.DataFrame({
    'Model': ['Random Forest', 'Gradient Boosting', 'Ridge', 'Elastic Net', 'Neural Network'],
    'Train_RMSE': [rf_train_rmse, gb_train_rmse, ridge_train_rmse, elastic_train_rmse, mlp_train_rmse],
    'Test_RMSE': [rf_test_rmse, gb_test_rmse, ridge_test_rmse, elastic_test_rmse, mlp_test_rmse],
    'Test_R2': [rf_test_r2, gb_test_r2, ridge_test_r2, elastic_test_r2, mlp_test_r2]
})

comparison['Overfit'] = comparison['Train_RMSE'] - comparison['Test_RMSE']
comparison = comparison.sort_values('Test_RMSE')

print(comparison.to_string(index=False))

# ============================================================================
# ENSEMBLE: WEIGHTED AVERAGE
# ============================================================================

print("\n" + "="*70)
print("ENSEMBLE: SIMPLE AVERAGE")
print("="*70)

ensemble_pred_test = (rf_pred_test + gb_pred_test + ridge_pred_test) / 3
ensemble_rmse = np.sqrt(mean_squared_error(y_test, ensemble_pred_test))
ensemble_r2 = r2_score(y_test, ensemble_pred_test)

print(f"Ensemble Test RMSE: {ensemble_rmse:.4f}")
print(f"Ensemble Test R²: {ensemble_r2:.4f}")

# ============================================================================
# VISUALIZE PREDICTIONS
# ============================================================================

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

models = [
    ('Random Forest', rf_pred_test),
    ('Gradient Boosting', gb_pred_test),
    ('Ridge', ridge_pred_test),
    ('Elastic Net', elastic_pred_test),
    ('Neural Network', mlp_pred_test),
    ('Ensemble', ensemble_pred_test)
]

for idx, (name, preds) in enumerate(models):
    ax = axes[idx]
    ax.scatter(y_test, preds, alpha=0.6)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    ax.set_xlabel('Actual')
    ax.set_ylabel('Predicted')
    ax.set_title(name)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('ml_predictions.png', dpi=300)
plt.close()

print("\nPrediction plots saved as 'ml_predictions.png'")

# ============================================================================
# APPLY TO NEW GROUP
# ============================================================================

print("\n" + "="*70)
print("APPLYING TO NEW GROUP")
print("="*70)

def predict_new_group(new_group_df, model, scaler=None, scale=False):
    """
    new_group_df: DataFrame with 'time' and 'variable' columns
    model: trained sklearn model
    scaler: fitted StandardScaler (if scaling needed)
    scale: whether to scale features
    """
    new_features = create_features(
        new_group_df.assign(group=999), 
        max_lag=2
    )
    
    X_new = new_features[feature_cols]
    
    if scale and scaler:
        X_new = scaler.transform(X_new)
    
    predictions = model.predict(X_new)
    
    new_features['prediction'] = predictions
    return new_features[['time', 'variable', 'prediction']]

print("""
Example usage:

# Load new group data
new_group = pd.DataFrame({
    'time': [1, 2, 3, 4, 5, 6, 7, 8],
    'variable': [10.2, 11.1, 10.8, 12.3, 13.1, 12.9, 14.0, 14.5]
})

# Make predictions
predictions = predict_new_group(new_group, rf_model, scale=False)

# For models needing scaling (Ridge, Elastic Net, MLP)
predictions = predict_new_group(new_group, ridge_model, scaler, scale=True)
""")

# ============================================================================
# RECOMMENDATIONS
# ============================================================================

print("\n" + "="*70)
print("RECOMMENDATIONS")
print("="*70)

print("""
Best ML Approaches for Your Case:

1. GRADIENT BOOSTING (Recommended)
   ✓ Handles non-linearity well
   ✓ Good with limited data
   ✓ Less prone to overfitting than Random Forest
   ✓ Built-in feature importance

2. RANDOM FOREST
   ✓ Robust to outliers
   ✓ Handles feature interactions
   ✗ Can overfit with small data
   
3. RIDGE/ELASTIC NET
   ✓ Interpretable coefficients
   ✓ Regularization prevents overfitting
   ✓ Works well as baseline
   ✗ Assumes linear relationships

4. ENSEMBLE
   ✓ Often best performance
   ✓ Reduces model-specific errors
   
Key Feature Engineering Tips:
- Lag features (1-3 lags) are most important
- Rolling statistics capture recent trends
- Time and time² capture non-linear trends
- Differences capture changes

With 8 periods per group:
- Keep models simple (low max_depth, regularization)
- Feature engineering matters more than model complexity
- Cross-validation is critical
""")

Feature engineering complete
Original data: 240 rows
After feature engineering: 180 rows

Features: ['time', 'lag1', 'lag2', 'roll_mean_3', 'roll_std_3', 'time_squared', 'diff1', 'diff2']

Train size: 120, Test size: 60

MODEL 1: RANDOM FOREST
Train RMSE: 0.0744
Test RMSE: 0.3015
Test R²: 0.6957

Top 5 features:
    feature  importance
      diff1    0.492027
      diff2    0.303916
 roll_std_3    0.094969
roll_mean_3    0.081734
       lag1    0.019239

MODEL 2: GRADIENT BOOSTING
Train RMSE: 0.0149
Test RMSE: 0.2915
Test R²: 0.7156

MODEL 3: RIDGE REGRESSION
Train RMSE: 0.0030
Test RMSE: 0.0033
Test R²: 1.0000

Top 5 coefficients:
    feature  coefficient
roll_mean_3     0.200210
      diff1     0.197744
      diff2     0.189807
       lag1     0.047978
       lag2     0.033293

MODEL 4: ELASTIC NET
Train RMSE: 0.0917
Test RMSE: 0.1297
Test R²: 0.9437
Non-zero coefficients: 3/8

MODEL 5: NEURAL NETWORK (MLP)
Train RMSE: 0.0265
Test RMSE: 0.1134
Test R²: 0.9569

MODEL COMPARISON
      

## Modular Code

In [ ]:
from abc import ABC, abstractmethod
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

class TimeSeriesModel(ABC):
    """Base class for all time series models"""
    
    def __init__(self, name: str):
        self.name = name
        self.model = None
        self.metrics = {}
    
    @abstractmethod
    def fit(self, X, y):
        """Fit the model"""
        pass
    
    @abstractmethod
    def predict(self, X):
        """Make predictions"""
        pass
    
    def evaluate(self, y_true, y_pred):
        """Calculate model performance metrics"""
        self.metrics = {
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2_score(y_true, y_pred)
        }
        return self.metrics

In [ ]:
import pandas as pd

def create_lagged_features(df: pd.DataFrame, group_col: str, 
                         time_col: str, target_col: str, max_lag: int = 3):
    """Create lagged features for time series data"""
    df_features = []
    
    for g in df[group_col].unique():
        group_df = df[df[group_col] == g].copy().sort_values(time_col)
        
        # Lagged values
        for i in range(1, max_lag + 1):
            group_df[f'lag{i}'] = group_df[target_col].shift(i)
        
        # Rolling statistics
        group_df['roll_mean_3'] = group_df[target_col].rolling(window=3).mean()
        group_df['roll_std_3'] = group_df[target_col].rolling(window=3).std()
        
        # Time features
        group_df['time_squared'] = group_df[time_col] ** 2
        
        # Differences
        group_df['diff1'] = group_df[target_col].diff(1)
        group_df['diff2'] = group_df[target_col].diff(2)
        
        df_features.append(group_df)
    
    result = pd.concat(df_features, ignore_index=True)
    return result.dropna()

In [ ]:
# from .base import TimeSeriesModel
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd
import numpy as np

class ARIMAModel(TimeSeriesModel):
    def __init__(self, order=(1,0,0)):
        super().__init__(name='ARIMA')
        self.order = order
        self.params = {}
    
    def fit(self, data: pd.DataFrame, group_col: str, target_col: str):
        """Fit ARIMA models for each group"""
        group_params = []
        
        for g in data[group_col].unique():
            group_data = data[data[group_col] == g][target_col].values
            try:
                model = ARIMA(group_data, order=self.order)
                fit = model.fit()
                group_params.append({
                    'group': g,
                    'ar_coef': fit.params[1],
                    'intercept': fit.params[0],
                    'sigma': np.sqrt(fit.params[2])
                })
            except:
                continue
        
        self.params = pd.DataFrame(group_params)
        return self

In [ ]:
from .base import TimeSeriesModel
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler

class GradientBoostingTS(TimeSeriesModel):
    def __init__(self, **kwargs):
        super().__init__(name='GradientBoosting')
        self.model = GradientBoostingRegressor(**kwargs)
        self.scaler = StandardScaler()
    
    def fit(self, X, y):
        X_scaled = self.scaler.fit_transform(X)
        self.model.fit(X_scaled, y)
        return self
    
    def predict(self, X):
        X_scaled = self.scaler.transform(X)
        return self.model.predict(X_scaled)

In [ ]:
from utils.feature_engineering import create_lagged_features
from models.statistical import ARIMAModel
from models.machine_learning import GradientBoostingTS
import pandas as pd

def main():
    # Load and prepare data
    df = pd.read_csv('data/processed_data.csv')
    
    # Create features
    df_features = create_lagged_features(
        df, 
        group_col='Player',
        time_col='Wk',
        target_col='xG',
        max_lag=3
    )
    
    # Train/test split
    train_mask = df_features['Wk'] <= 6
    
    # Train models
    models = {
        'arima': ARIMAModel(order=(1,0,0)),
        'gbm': GradientBoostingTS(
            n_estimators=100,
            max_depth=3,
            learning_rate=0.1
        )
    }
    
    # Train and evaluate each model
    results = {}
    for name, model in models.items():
        model.fit(...)  # Add your training logic
        predictions = model.predict(...)  # Add prediction logic
        results[name] = model.evaluate(y_true, predictions)

if __name__ == '__main__':
    main()